In [1]:
from sklearn.metrics import accuracy_score, classification_report , confusion_matrix , ConfusionMatrixDisplay
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pickle as pkl
import pandas as pd

<h1 style='text-align:center'>Load Dataset and Models</h1>

In [2]:
test_df = pd.read_csv('./test.csv')

In [4]:
test_df

,id,path,pred_1,pred_2,pred_3
0,0,imageset/test/0.JPEG,3,0,7
1,1,imageset/test/1.JPEG,0,0,0
2,2,imageset/test/2.JPEG,0,0,0
3,3,imageset/test/3.JPEG,0,0,0
4,4,imageset/test/4.JPEG,0,0,4
...,...,...,...,...,...
3920,3920,imageset/test/3920.JPEG,9,9,9
3921,3921,imageset/test/3921.JPEG,9,9,5
3922,3922,imageset/test/3922.JPEG,9,9,9
3923,3923,imageset/test/3923.JPEG,9,9,9


In [15]:
test_images = pkl.load(open('./test_images.pickle','rb'))

In [ ]:
test_images.shape

In [6]:
# Efficient_model = load_model('./Models/EfficientNetV2B0_custom.h5')
# ResNet_model = load_model('./Models/ResNet101V2_custom.h5')
custom_model = load_model('./Models/C20_custom.h5')

<h1>Validating Accuracy</h1>

In [ ]:
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.15)

In [ ]:
train_generator = datagen.flow_from_directory(
    directory="./imageset/train/",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="sparse",
    target_size=(224, 224),
    subset="training"
)
validation_generator = datagen.flow_from_directory(
    directory="./imageset/train/",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="sparse",
    target_size=(224, 224),
    subset="validation"  # Specify that this is the validation set
)

In [ ]:
STEP_SIZE_TEST=validation_generator.n//validation_generator.batch_size+1
validation_generator.reset()
y_pred_1=Efficient_model.predict(validation_generator)
y_pred_2=ResNet_model.predict(validation_generator)
y_pred_3=custom_model.predict(validation_generator)
# y_pred_classes =  np.argmax(np.bincount([y_pred_1, y_pred_2, y_pred_3], minlength=10))
y_true = validation_generator.classes


# Compute confusion matrix
# cm = confusion_matrix(y_true, y_pred_classes)

In [ ]:
y_pred_1 = np.argmax(y_pred_1,axis=1)
y_pred_2 = np.argmax(y_pred_2,axis=1)
y_pred_3 = np.argmax(y_pred_3,axis=1)

In [ ]:
voting_predictions = np.array([y_pred_1, y_pred_2, y_pred_3])
y_pred_classes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=voting_predictions)
cm = confusion_matrix(y_true, y_pred_classes)


In [ ]:
print(f"Accuracy: {accuracy_score(y_true, y_pred_classes)}")
print(f"Classwise Classification Report: \n{classification_report(y_true, y_pred_classes)}")
con_matrix = confusion_matrix(np.ravel(y_true), y_pred_classes)
print("\nConfusion Matrix: ")
con_matrix = ConfusionMatrixDisplay(con_matrix)
con_matrix.plot()
plt.show()

<h1>Predicting test labels</h1>

In [7]:
# pred=Efficient_model.predict(test_images)
# pred=ResNet_model.predict(test_images)
pred=custom_model.predict(test_images)

123/123 [==============================] - 13s 55ms/step


In [8]:
pred = np.argmax(pred,axis=1)

In [9]:
test_df['pred_3'] = pred

In [10]:
test_df

,id,path,pred_1,pred_2,pred_3
0,0,imageset/test/0.JPEG,3,0,7
1,1,imageset/test/1.JPEG,0,0,0
2,2,imageset/test/2.JPEG,0,0,0
3,3,imageset/test/3.JPEG,0,0,0
4,4,imageset/test/4.JPEG,0,0,4
...,...,...,...,...,...
3920,3920,imageset/test/3920.JPEG,9,9,9
3921,3921,imageset/test/3921.JPEG,9,9,5
3922,3922,imageset/test/3922.JPEG,9,9,9
3923,3923,imageset/test/3923.JPEG,9,9,9


In [11]:
test_df.to_csv('test.csv',index=False)

<h1>Voting</h1>

In [5]:
pred_1 = test_df['pred_1']
pred_2 = test_df['pred_2']
pred_3 = test_df['pred_3']

In [6]:
voting_predictions = np.array([pred_1, pred_2, pred_3])
y_pred_classes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=voting_predictions)

In [ ]:
test_df['label'] = y_pred_classes